In [1]:
using WignerFamilies, CSV, DataFrames, WignerSymbols, Distributed
import Base.Threads.@threads

### Input parameters

In [2]:
parity="even"
include_partial_triangles=false

min_l = 2
# Nl = 6
# dl = 20;
Nl = 3
dl = 4;

### Load Cl input

In [3]:
Cl = Array(DataFrame(CSV.File("cl")));

### Compute normalization

In [37]:
function NormElement(bins,binsp,prefs)
    
    # Define bins
    bin1,bin2,bin3,bin4,binL = bins
    bin1p,bin2p,bin3p,bin4p,binLp = binsp
    pref1,pref2,pref3 = prefs
    
    # Iterate over ell
    value_even = 0
    value_odd = 0
    for l1 in min_l+bin1*dl:min_l+(bin1+1)*dl
        for l2 in min_l+bin2*dl:min_l+(bin2+1)*dl

            # compute 3j symbol with spin (-1,-1,2)
            tj12_arr = WignerFamilies.wigner3j_f(l1,l2,-1,-1)

            for l3 in min_l+bin3*dl:min_l+(bin3+1)*dl

                tj23_arr = WignerFamilies.wigner3j_f(l2,l3,-1,-1)
                tj13_arr = WignerFamilies.wigner3j_f(l1,l3,-1,-1)

                for l4 in min_l+bin4*dl:min_l+(bin4+1)*dl

                    tj14_arr = WignerFamilies.wigner3j_f(l1,l4,-1,-1)
                    tj24_arr = WignerFamilies.wigner3j_f(l2,l4,-1,-1)
                    tj34_arr = WignerFamilies.wigner3j_f(l3,l4,-1,-1)

                    # Continue if wrong-parity, or in [b1=b3, b2=b4] bin and odd
                    even = ((-1)^(l1+l2+l3+l4)==1)
                    if (~even) & ((bin1==bin3)&(bin2==bin4))
                        continue
                    end
                    if (~even) & ((bin1p==bin3p)&(bin2p==bin4p))
                        continue
                    end
                    if (even) & (parity=="odd")
                        continue
                    end
                    if (~even) & (parity=="even")
                       continue
                    end

                    # Iterate over L 
                    for L in min_l+binL*dl:min_l+(binL+1)*dl

                        # Check triangle conditions
                        if (L<abs(l1-l2)) | (L>l1+l2) | (L<abs(l3-l4)) | (L>l3+l4)
                            continue
                        end

                        tj1234 = tj12_arr[L]*tj34_arr[L]
                        
                        if tj1234==0
                            continue
                        end
                        
                        prefactor = (2*l1+1)*(2*l2+1)*(2*l3+1)*(2*l4+1)*(2*L+1)/Cl[l1]/Cl[l2]/Cl[l3]/Cl[l4]/(4*pi)^2

                        # Add first permutation
                        if (pref1!=0)
                            if ~even
                                value_odd  += pref1*tj1234^2*prefactor
                            else
                                value_even += pref1*tj1234^2*prefactor
                            end
                        end
                        
                        # Iterate over L' for off-diagonal terms
                        for Lp in min_l+binLp*dl:min_l+(binLp+1)*dl

                            # Impose 6j symmetries
                            if (Lp<abs(l3-l4)) | (Lp>l3+l4) | (Lp<abs(l1-l2)) | (Lp>l1+l2)
                                continue
                            end

                            # Compute 3j symbols if non-zero
                            if (Lp<abs(l1-l3)) | (Lp>l1+l3) | (Lp<abs(l2-l4)) | (Lp>l2+l4)
                                tj1324=0
                            else
                                tj1324 = tj13_arr[Lp]*tj24_arr[Lp]
                            end
                            if (Lp<abs(l1-l4)) | (Lp>l1+l4) | (Lp<abs(l2-l3)) | (Lp>l2+l3)
                                tj1432 = 0
                            else
                                tj1432 = tj14_arr[Lp]*tj23_arr[Lp]
                            end

                            ## add second permutation
                            if (pref2!=0) & (tj1324!=0)
                                if ~even
                                    value_odd  += pref2*(-1)^(l2+l3)*tj1234*tj1324*WignerSymbols.wigner6j(L,l1,l2,Lp,l4,l3)*prefactor*(2*Lp+1)
                                else
                                    value_even += pref2*(-1)^(l2+l3)*tj1234*tj1324*WignerSymbols.wigner6j(L,l1,l2,Lp,l4,l3)*prefactor*(2*Lp+1)
                                end
                            end

                            ## add third permutation
                            if (pref3!=0) & (tj1432!=0)
                                if ~even
                                    value_odd  += pref3*(-1.)^(L+Lp)*tj1234*tj1432*WignerSymbols.wigner6j(L,l1,l2,Lp,l3,l4)*prefactor*(2*Lp+1)
                                else
                                    value_even += pref3*(-1.)^(L+Lp)*tj1234*tj1432*WignerSymbols.wigner6j(L,l1,l2,Lp,l3,l4)*prefactor*(2*Lp+1)
                                end
                            end
                        end
                    end   
                end
            end
        end
    end
    value_even, value_odd
end;

In [22]:
function check_bin(bin1,bin2,bin3)
    if include_partial_triangles
        good = 0
        for l1 in min_l+bin1*dl:min_l+(bin1+1)*dl
            for l2 in min_l+bin2*dl:min_l+(bin2+1)*dl
                for l3 in min_l+bin3*dl:min_l+(bin3+1)*dl
                    if (l1>=abs(l1-l2))&(l3<=l1+l2)
                        good = 1
                    end
                    if good==1
                        break
                    end
                end
                if good==1
                    break
                end
            end
            if good==1
                break
            end
        end
        if good==1
            return true
        else
            return false
        end
    else
        l1 = min_l+(bin1+0.5)*dl
        l2 = min_l+(bin2+0.5)*dl
        l3 = min_l+(bin3+0.5)*dl
        if (l3<abs(l1-l2))|(l3>l1+l2)
            return false
        else
            return true
        end
    end
end;

### Compute list of inputs

In [33]:
# compute symmetry factors
function symmetry_factor(bin1,bin2,bin3,bin4)
    if (bin1==bin2)&(bin3==bin4)&(bin1==bin3)
        return 8
    elseif (bin1==bin2)&(bin3==bin4)
        return 4
    elseif bin1==bin2
        return 2
    elseif bin3==bin4
        return 2
    elseif (bin1==bin3)&(bin2==bin4)
        return 2
    else
        return 1
    end
end;

In [34]:
biases = []
biasesp = []
prefs = []
indices = []

index1e = 0
index1o = 0
for bin1 in 0:Nl-1
    for bin2 in bin1:Nl-1
        for bin3 in bin1:Nl-1
            for bin4 in bin3:Nl-1
                if (bin1==bin3)&(bin4<bin2)
                    continue
                end
                if (bin1==bin3)&(bin2==bin4)&(parity=="odd")
                    continue
                end
                for binL in 0:Nl-1
                    # skip bins outside the triangle conditions
                    if ~ check_bin(bin1,bin2,binL)
                        continue
                    end
                    if ~ check_bin(bin3,bin4,binL)
                        continue
                    end

                    # Update indices
                    index1e += 1
                    if ~((bin1==bin3)&(bin2==bin4))
                        index1o += 1 # no equal bins!
                    end
                    
                    # Iterate over second set of bins
                    index2e = 0
                    index2o = 0
                    for bin1p in 0:Nl-1
                        for bin2p in bin1p:Nl-1
                            for bin3p in bin1p:Nl-1
                                for bin4p in bin3p:Nl-1
                                    if (bin1p==bin3p)&(bin4p<bin2p)
                                        continue
                                    end
                                    if (bin1p==bin3p)&(bin2p==bin4p)&(parity=="odd")
                                        continue
                                    end
                                    for binLp in 0:Nl-1
                                        # skip bins outside the triangle conditions
                                        if ~check_bin(bin1p,bin2p,binLp)
                                            continue
                                        end
                                        if ~check_bin(bin3p,bin4p,binLp)
                                            continue
                                        end

                                        # Update indices
                                        index2e += 1
                                        if ~((bin1p==bin3p)&(bin2p==bin4p))
                                            index2o += 1 # no equal bins!
                                        end
                                        
                                        ## Compute permutation factors
                                        pref1  = (bin1==bin1p)*(bin2==bin2p)*(bin3==bin3p)*(bin4==bin4p)*(binL==binLp)
                                        pref1 += (bin1==bin2p)*(bin2==bin1p)*(bin3==bin3p)*(bin4==bin4p)*(binL==binLp)
                                        pref1 += (bin1==bin1p)*(bin2==bin2p)*(bin3==bin4p)*(bin4==bin3p)*(binL==binLp)
                                        pref1 += (bin1==bin2p)*(bin2==bin1p)*(bin3==bin4p)*(bin4==bin3p)*(binL==binLp)
                                        pref1 += (bin1==bin3p)*(bin2==bin4p)*(bin3==bin1p)*(bin4==bin2p)*(binL==binLp)
                                        pref1 += (bin1==bin4p)*(bin2==bin3p)*(bin3==bin1p)*(bin4==bin2p)*(binL==binLp)
                                        pref1 += (bin1==bin3p)*(bin2==bin4p)*(bin3==bin2p)*(bin4==bin1p)*(binL==binLp)
                                        pref1 += (bin1==bin4p)*(bin2==bin3p)*(bin3==bin2p)*(bin4==bin1p)*(binL==binLp)

                                        pref2  = (bin1==bin1p)*(bin2==bin3p)*(bin3==bin2p)*(bin4==bin4p)
                                        pref2 += (bin1==bin2p)*(bin2==bin3p)*(bin3==bin1p)*(bin4==bin4p)
                                        pref2 += (bin1==bin1p)*(bin2==bin4p)*(bin3==bin2p)*(bin4==bin3p)
                                        pref2 += (bin1==bin2p)*(bin2==bin4p)*(bin3==bin1p)*(bin4==bin3p)
                                        pref2 += (bin1==bin3p)*(bin2==bin1p)*(bin3==bin4p)*(bin4==bin2p)
                                        pref2 += (bin1==bin3p)*(bin2==bin2p)*(bin3==bin4p)*(bin4==bin1p)
                                        pref2 += (bin1==bin4p)*(bin2==bin1p)*(bin3==bin3p)*(bin4==bin2p)
                                        pref2 += (bin1==bin4p)*(bin2==bin2p)*(bin3==bin3p)*(bin4==bin1p)

                                        pref3  = (bin1==bin1p)*(bin2==bin4p)*(bin3==bin3p)*(bin4==bin2p)
                                        pref3 += (bin1==bin2p)*(bin2==bin4p)*(bin3==bin3p)*(bin4==bin1p)
                                        pref3 += (bin1==bin1p)*(bin2==bin3p)*(bin3==bin4p)*(bin4==bin2p)
                                        pref3 += (bin1==bin2p)*(bin2==bin3p)*(bin3==bin4p)*(bin4==bin1p)
                                        pref3 += (bin1==bin3p)*(bin2==bin2p)*(bin3==bin1p)*(bin4==bin4p)
                                        pref3 += (bin1==bin3p)*(bin2==bin1p)*(bin3==bin2p)*(bin4==bin4p)
                                        pref3 += (bin1==bin4p)*(bin2==bin2p)*(bin3==bin1p)*(bin4==bin3p)
                                        pref3 += (bin1==bin4p)*(bin2==bin1p)*(bin3==bin2p)*(bin4==bin3p)

                                        if pref1+pref2+pref3==0
                                            continue
                                        end
                                        
                                        push!(biases,[bin1,bin2,bin3,bin4,binL])
                                        push!(biasesp,[bin1p,bin2p,bin3p,bin4p,binLp])
                                        push!(prefs,[pref1,pref2,pref3]) 
                                        push!(indices,[index1e,index1o,index2e,index2o])
                                    end
                                end
                            end
                        end
                    end
                end
            end
        end
    end
end

### Assemble Fisher

In [24]:
fish_odd = zeros(index1o,index1o);
fish_even = zeros(index1e,index1e);
max_el = length(biases)

@time begin
    @threads for i in 1:length(biases)

        print("Analyzing element $i of $max_el")
        println("")

        # Read attributes
        bin1,bin2,bin3,bin4,binL = biases[i]
        bin1p,bin2p,bin3p,bin4p,binLp = biasesp[i]
        pref1,pref2,pref3 = prefs[i]
        i1e,i1o,i2e,i2o = indices[i]

        if (i2e<i1e) | (i2o<i1o)
            continue
        end
        
        # Compute symmetries
        sym  = symmetry_factor(bin1,bin2,bin3,bin4)
        symp = symmetry_factor(bin1p,bin2p,bin3p,bin4p)

        value_even, value_odd = NormElement([bin1,bin2,bin3,bin4,binL],[bin1p,bin2p,bin3p,bin4p,binLp],[pref1,pref2,pref3])

        # Add to output
        if (parity!="even")&(~((bin1==bin3)&(bin2==bin4)))&(~((bin1p==bin3p)&(bin2p==bin4p)))
            fish_odd[i1o, i2o] = value_odd/sym/symp
            fish_odd[i2o, i1o] = value_odd/sym/symp
        end
        if parity!="odd"
            fish_even[i1e, i2e] = value_even/sym/symp
            fish_even[i2e, i1e] = value_even/sym/symp
        end
    end
end

Analyzing element 1 of 181
Analyzing element 2 of 181
Analyzing element 3 of 181
Analyzing element 4 of 181
Analyzing element 5 of 181
Analyzing element 6 of 181
Analyzing element 7 of 181
Analyzing element 8 of 181
Analyzing element 9 of 181
Analyzing element 10 of 181
Analyzing element 11 of 181
Analyzing element 12 of 181
Analyzing element 13 of 181
Analyzing element 14 of 181
Analyzing element 15 of 181
Analyzing element 16 of 181
Analyzing element 17 of 181
Analyzing element 18 of 181
Analyzing element 19 of 181
Analyzing element 20 of 181
Analyzing element 21 of 181
Analyzing element 22 of 181
Analyzing element 23 of 181
Analyzing element 24 of 181
Analyzing element 25 of 181
Analyzing element 26 of 181
Analyzing element 27 of 181
Analyzing element 28 of 181
Analyzing element 29 of 181
Analyzing element 30 of 181
Analyzing element 31 of 181
Analyzing element 32 of 181
Analyzing element 33 of 181
Analyzing element 34 of 181
Analyzing element 35 of 181
Analyzing element 36 of 181
A

## add type casting?

In [28]:
if parity!="even"
    CSV.write("fish_odd_$(min_l)_$(dl)_$(Nl).csv",  Tables.table(fish_odd), writeheader=false, delim="\t")
end
if parity!="odd"
    CSV.write("fish_even_$(min_l)_$(dl)_$(Nl).csv",  Tables.table(fish_even), writeheader=false, delim="\t")
end

println("Complete!");

Complete!
